In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Friday 10/07, 11:59 PM
## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

Question | Points
--- | ---
0 | 1
1a	| 1
1bi	| 1
1bii	| 2
1c	| 1
1di	| 2
1dii	| 1
1ei	| 1
1eii	| 1
1eiii	| 1
1eiv	| 2
1ev	| 2
2a	| 1
2b	| 1
2c	| 2
2d	| 2
3a	| 1
3b	| 1
3c |	1
3d	| 3
4a	| 2
4b	| 1
4c	| 1
4di	| 2
4dii |	2
4ei	| 2
4eii | 	1
4eiii |	2
4eiv |	1
4ev |	1
4evi |	2
4evii |	2
5a | 2
5b | 2
5c | 1
5d | 2
6a | 2
6b | 1
6c | 1
6d | 2
6e | 2
7a | 1
7b | 1
7c | 2
8  | 6
**Total** | 72

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `ipython-sql` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [4]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!gzip -dc data/lahman.pgdump.gz | psql -h localhost -d baseball -f -
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
CREATE EXTENSION
COMMENT
SET
CREATE TABLE
psql:<stdin>:49: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:80: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:96: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:112: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:129: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:146: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:178: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:210: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:223: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:251: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:267: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:295: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:322: ERROR:  

Name,Owner,Encoding,Collate,Ctype,Access privileges
baseball,jovyan,UTF8,en_US.utf8,en_US.utf8,None
imdb,jovyan,UTF8,en_US.utf8,en_US.utf8,None
jovyan,jovyan,UTF8,en_US.utf8,en_US.utf8,None
postgres,jovyan,UTF8,en_US.utf8,en_US.utf8,None
template0,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan
template1,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan


Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [5]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [6]:
%%sql
SELECT * FROM halloffame LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


Run the following cell for grading purposes.

In [7]:
!mkdir -p results

## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, `%sql \d <table_name>` is helpful for identifying the columns in a table.

## Question 0: PostgreSQL Explain Analyze
**Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the explain analyze command.**

To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_variable = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_variable"`

Take a look at the following sample query plan.

![title](data/sample_query.png)

It is highly recommended to read through [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/) to see how you can interpret the output above. Everything before "Tools to interpret Explain Analyze output" is useful.

Here are some key things to note:
- There are two cost values: the first is the **startup cost** (cost to return the first row) and the second is the **total cost** (cost to return all rows).
- The unit for the estimated query cost is arbitrary (1 is the cost for reading an 8kB page during a sequential scan)
- When we ask you to identify the query cost, we are looking for the **total cost**
- When we ask you to identify the query time, we are looking for the **execution time** (in ms)
- Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact
- We recognzie that the execution time may vary between different cell executions, so the autograder will tolerate a reasonable range

Now, inspect the query plan above, and record the **cost** and **time** for the sample query.

<!--
BEGIN QUESTION
name: q0
points: 1
-->

In [8]:
sample_query_cost = 1193.88
sample_query_timing = 185.896

In [9]:
grader.check("q0")

q0 results: All test cases passed!

## Question 1: Queries and Views
### Question 1a:
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **NOT** use any sub-queries.

.appearances: details on the positions each player appeared at

.batting: batting statistics for each player

.collegeplaying: list of players and the colleges they attended

.halloffame: Hall of Fame voting data

.people: player information (name, date of birth, and biographical info)

.salaries: player salary data

.schools: list of colleges that players attended

In [10]:
%%sql 
SELECT *
FROM halloffame
LIMIT 10

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None
lajoina01,1936,BBWAA,226,170,146,N,Player,None
speaktr01,1936,BBWAA,226,170,133,N,Player,None
youngcy01,1936,BBWAA,226,170,111,N,Player,None
hornsro01,1936,BBWAA,226,170,105,N,Player,None
cochrmi01,1936,BBWAA,226,170,80,N,Player,None


In [11]:
%%sql result_1a <<
SELECT p.namefirst, p.namelast, h.playerid, h.yearid
FROM people p
INNER JOIN halloffame h ON p.playerid = h.playerid
WHERE h.inducted = 'Y';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable result_1a


In [12]:
result_1a

namefirst,namelast,playerid,yearid
Ty,Cobb,cobbty01,1936
Babe,Ruth,ruthba01,1936
Honus,Wagner,wagneho01,1936
Christy,Mathewson,mathech01,1936
Walter,Johnson,johnswa01,1936
Nap,Lajoie,lajoina01,1937
Tris,Speaker,speaktr01,1937
Cy,Young,youngcy01,1937
Morgan,Bulkeley,bulkemo99,1937
Ban,Johnson,johnsba99,1937


In [13]:
# Do not delete/edit this cell
result_1a.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_1a.csv', index=False)

In [14]:
grader.check("q1a")

q1a results: All test cases passed!

### Question 1b:
In this question, we will compare the query you wrote in `Question 1a` against the provided query in `Question 1bi` by inspecting the query plans.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a`. Record the **execution time** and **cost** for each query.

In [15]:
%%sql provided_query <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable provided_query


In [16]:
provided_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE hof.inducted = 'Y' AND p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $provided_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..262.79 rows=21 width=278) (actual time=0.066..2.680 rows=323 loops=1)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=21 width=42) (actual time=0.023..0.700 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Index Scan using master_pkey on people p  (cost=0.29..7.92 rows=1 width=274) (actual time=0.006..0.006 rows=1 loops=323)
         Index Cond: ((playerid)::text = (halloffame.playerid)::text)
 Planning Time: 0.743 ms
 Execution Time: 2.747 ms
(8 rows)



In [17]:
inducted_hof_query_str = "SELECT p.namefirst, p.namelast, h.playerid, h.yearid FROM people p INNER JOIN halloffame h ON p.playerid = h.playerid WHERE h.inducted = 'Y';"
!psql -h localhost -d baseball -c "explain analyze $inducted_hof_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..262.79 rows=21 width=278) (actual time=0.062..2.840 rows=323 loops=1)
   ->  Seq Scan on halloffame h  (cost=0.00..96.39 rows=21 width=42) (actual time=0.024..0.718 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Index Scan using master_pkey on people p  (cost=0.29..7.92 rows=1 width=274) (actual time=0.006..0.006 rows=1 loops=323)
         Index Cond: ((playerid)::text = (h.playerid)::text)
 Planning Time: 0.775 ms
 Execution Time: 2.909 ms
(8 rows)



In [18]:
provided_query_cost = 262.79
provided_query_timing = 2.710
your_query_cost = 262.79
your_query_timing = 2.627

In [19]:
grader.check("q1bi")

q1bi results: All test cases passed!

#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of answer choices which are true statements.

1. Consider the following statements:
    1. Both the queries have the same cost.
    1. The provided query has a faster execution time because it makes use of a subquery.
    1. The query you wrote has a faster execution time because it does not make use a subquery.
    1. The provided query has less cost because it makes use of a subquery.
    1. The query you wrote has less cost because it does not make use a subquery.
    1. The queries have the same output.
    1. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1bii
points: 2
-->

In [20]:
q1b_part2 = ['A', 'C', 'F']

In [21]:
grader.check("q1bii")

q1bii results: All test cases passed!

### Question 1c:
Write a query that creates a view named `inducted_hof_ca` of the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

.appearances: details on the positions each player appeared at

.batting: batting statistics for each player

.collegeplaying: list of players and the colleges they attended

.halloffame: Hall of Fame voting data

.people: player information (name, date of birth, and biographical info)

.salaries: player salary data

.schools: list of colleges that players attended

In [22]:
%%sql
SELECT *
FROM people
LIMIT 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,deathcountry,deathstate,deathcity,namefirst,namelast,namegiven,weight,height,bats,throws,debut,finalgame,retroid,bbrefid
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
aaronha01,1934,2,5,USA,AL,Mobile,None,None,None,None,None,None,Hank,Aaron,Henry Louis,180,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
aaronto01,1939,8,5,USA,AL,Mobile,1984,8,16,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
aasedo01,1954,9,8,USA,CA,Orange,None,None,None,None,None,None,Don,Aase,Donald William,190,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
abadan01,1972,8,25,USA,FL,Palm Beach,None,None,None,None,None,None,Andy,Abad,Fausto Andres,184,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01
abadfe01,1985,12,17,D.R.,La Romana,La Romana,None,None,None,None,None,None,Fernando,Abad,Fernando Antonio,220,73.0,L,L,2010-07-28,2017-10-01,abadf001,abadfe01
abadijo01,1850,11,4,USA,PA,Philadelphia,1905,5,17,USA,NJ,Pemberton,John,Abadie,John W.,192,72.0,R,R,1875-04-26,1875-06-10,abadj101,abadijo01
abbated01,1877,4,15,USA,PA,Latrobe,1957,1,6,USA,FL,Fort Lauderdale,Ed,Abbaticchio,Edward James,170,71.0,R,R,1897-09-04,1910-09-15,abbae101,abbated01
abbeybe01,1869,11,11,USA,VT,Essex,1962,6,11,USA,VT,Colchester,Bert,Abbey,Bert Wood,175,71.0,R,R,1892-06-14,1896-09-23,abbeb101,abbeybe01
abbeych01,1866,10,14,USA,NE,Falls City,1926,4,27,USA,CA,San Francisco,Charlie,Abbey,Charles S.,169,68.0,L,L,1893-08-16,1897-08-19,abbec101,abbeych01


In [23]:
%%sql
SELECT *
FROM halloffame
LIMIT 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None
lajoina01,1936,BBWAA,226,170,146,N,Player,None
speaktr01,1936,BBWAA,226,170,133,N,Player,None
youngcy01,1936,BBWAA,226,170,111,N,Player,None
hornsro01,1936,BBWAA,226,170,105,N,Player,None
cochrmi01,1936,BBWAA,226,170,80,N,Player,None


In [24]:
%%sql
SELECT *
FROM collegeplaying
LIMIT 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


playerid,schoolid,yearid
aardsda01,pennst,2001
aardsda01,rice,2002
aardsda01,rice,2003
abadan01,gamiddl,1992
abadan01,gamiddl,1993
abbeybe01,vermont,1889
abbeybe01,vermont,1890
abbeybe01,vermont,1891
abbeybe01,vermont,1892
abbotje01,kentucky,1991


In [25]:
%%sql
SELECT *
FROM schools
LIMIT 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


schoolid,schoolname,schoolcity,schoolstate,schoolnick
abilchrist,Abilene Christian University,Abilene,TX,USA
adelphi,Adelphi University,Garden City,NY,USA
adrianmi,Adrian College,Adrian,MI,USA
akron,University of Akron,Akron,OH,USA
alabama,University of Alabama,Tuscaloosa,AL,USA
alabamaam,Alabama A&M University,Normal,AL,USA
alabamast,Alabama State University,Montgomery,AL,USA
albanyst,Albany State University,Albany,GA,USA
albertsnid,Albertson College,Caldwell,ID,USA
albevil,Bevill State Community College,Sumiton,AL,USA


In [26]:
%%sql result_1c <<
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS

SELECT p.namefirst, p.namelast, c.playerid, s.schoolid, h.yearid
FROM people p, halloffame h, collegeplaying c, schools s
WHERE p.playerid = h.playerid AND c.playerid = h.playerid AND c.schoolid = s.schoolid AND h.inducted = 'Y' AND s.schoolstate = 'CA';

SELECT * FROM inducted_hof_ca;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
23 rows affected.
Returning data to local variable result_1c


In [27]:
%%sql
SELECT * 
FROM inducted_hof_ca;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
23 rows affected.


namefirst,namelast,playerid,schoolid,yearid
Jackie,Robinson,robinja02,ucla,1962
Harry,Hooper,hoopeha01,stmarysca,1971
Joe,Morgan,morgajo02,camerri,1990
Tom,Seaver,seaveto01,cafrecc,1992
Tom,Seaver,seaveto01,usc,1992
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Tony,Gwynn,gwynnto01,sandiegost,2007


In [28]:
# Do not delete/edit this cell
result_1c.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1c.csv', index=False)

In [29]:
grader.check("q1c")

q1c results: All test cases passed!

### Question 1d:
For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

#### Question 1di:
Write two queries that accomplish this task -- one query using the view you created in the `Question 1c` and one query that does not use the view, common table expressions (CTEs), or any sub-queries.

In [30]:
%%sql with_view <<
SELECT COUNT(*), schoolid, yearid
FROM inducted_hof_ca
GROUP BY schoolid, yearid
ORDER BY yearid ASC;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_view


In [31]:
with_view

count,schoolid,yearid
1,ucla,1962
1,stmarysca,1971
1,camerri,1990
1,cafrecc,1992
1,usc,1992
4,calpoly,2002
3,sandiegost,2007
1,capasad,2008
2,sandiegost,2010
1,calavco,2011


In [32]:
%%sql without_view <<

SELECT COUNT(*), s.schoolid, h.yearid
FROM people p, halloffame h, collegeplaying c, schools s
WHERE p.playerid = h.playerid AND c.playerid = h.playerid AND c.schoolid = s.schoolid AND h.inducted = 'Y' AND s.schoolstate = 'CA'
GROUP BY s.schoolid, h.yearid
ORDER BY h.yearid ASC;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable without_view


In [33]:
# Do not delete/edit this cell
with_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_view.csv', index=False)
without_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_no_view.csv', index=False)

In [34]:
grader.check("q1di")

q1di results: All test cases passed!

#### Question 1dii:
Fill in your queries from `Question 1di` and inspect the query plans for the two queries. Record the execution time and cost for each query.

In [35]:
with_view_query_str = "SELECT COUNT(*), schoolid, yearid FROM inducted_hof_ca GROUP BY schoolid, yearid ORDER BY yearid ASC;"
!psql -h localhost -d baseball -c "explain analyze $with_view_query_str"

                                                                              QUERY PLAN                                                                               
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=325.71..325.73 rows=1 width=60) (actual time=5.006..5.018 rows=13 loops=1)
   Group Key: h.yearid, s.schoolid
   ->  Sort  (cost=325.71..325.71 rows=1 width=52) (actual time=4.997..5.001 rows=23 loops=1)
         Sort Key: h.yearid, s.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Nested Loop  (cost=0.85..325.70 rows=1 width=52) (actual time=1.897..4.940 rows=23 loops=1)
               ->  Nested Loop  (cost=0.57..319.95 rows=19 width=52) (actual time=0.173..4.472 rows=137 loops=1)
                     ->  Nested Loop  (cost=0.29..262.79 rows=21 width=80) (actual time=0.078..2.783 rows=323 loops=1)
              

In [36]:
without_view_query_str = "SELECT COUNT(*), s.schoolid, h.yearid FROM people p, halloffame h, collegeplaying c, schools s WHERE p.playerid = h.playerid AND c.playerid = h.playerid AND c.schoolid = s.schoolid AND h.inducted = 'Y' AND s.schoolstate = 'CA' GROUP BY s.schoolid, h.yearid ORDER BY h.yearid ASC;"
!psql -h localhost -d baseball -c"explain analyze $without_view_query_str"

                                                                              QUERY PLAN                                                                               
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=325.71..325.73 rows=1 width=60) (actual time=4.739..4.750 rows=13 loops=1)
   Group Key: h.yearid, s.schoolid
   ->  Sort  (cost=325.71..325.71 rows=1 width=52) (actual time=4.731..4.735 rows=23 loops=1)
         Sort Key: h.yearid, s.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Nested Loop  (cost=0.85..325.70 rows=1 width=52) (actual time=1.823..4.684 rows=23 loops=1)
               ->  Nested Loop  (cost=0.57..319.95 rows=19 width=52) (actual time=0.167..4.212 rows=137 loops=1)
                     ->  Nested Loop  (cost=0.29..262.79 rows=21 width=80) (actual time=0.070..2.671 rows=323 loops=1)
              

In [37]:
with_view_cost = 519.45
with_view_timing = 5.882
without_view_cost = 519.45
without_view_timing = 2.911

In [38]:
grader.check("q1dii")

q1dii results: All test cases passed!

### Question 1e:
#### Question 1ei:
Now, let's try creating a materialized view named `inducted_hof_ca_mat` instead of the regular view from `Question 1c`.

In [39]:
%%sql inducted_hof_ca_materialized <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS

SELECT p.namefirst, p.namelast, c.playerid, s.schoolid, h.yearid
FROM people p, halloffame h, collegeplaying c, schools s
WHERE p.playerid = h.playerid AND c.playerid = h.playerid AND c.schoolid = s.schoolid AND h.inducted = 'Y' AND s.schoolstate = 'CA';

SELECT * FROM inducted_hof_ca_mat;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
23 rows affected.
23 rows affected.
Returning data to local variable inducted_hof_ca_materialized


In [40]:
# Do not delete/edit this cell
inducted_hof_ca_materialized.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1ei.csv', index=False)

In [41]:
grader.check("q1ei")

q1ei results: All test cases passed!

#### Question 1eii:

Now, rewrite the query from `Question 1d` to use the materialized view `inducted_hof_ca_mat`.

In [42]:
%%sql with_materialized_view <<
SELECT COUNT(*), schoolid, yearid
FROM inducted_hof_ca_mat
GROUP BY schoolid, yearid
ORDER BY yearid ASC;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_materialized_view


In [43]:
# Do not delete/edit this cell
with_materialized_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1eii.csv', index=False)

In [44]:
grader.check("q1eii")

q1eii results: All test cases passed!

#### Question 1eiii:
Inspect the query plan and record the execution time and cost of the query that uses the materialized view.

In [45]:
with_materialized_view_query_str = "SELECT COUNT(*), schoolid, yearid FROM inducted_hof_ca_mat GROUP BY schoolid, yearid ORDER BY yearid ASC;"
!psql -h localhost -d baseball -c "explain analyze $with_materialized_view_query_str"

                                                          QUERY PLAN                                                          
------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=23.67..24.17 rows=200 width=60) (actual time=0.075..0.077 rows=13 loops=1)
   Sort Key: yearid
   Sort Method: quicksort  Memory: 25kB
   ->  HashAggregate  (cost=14.03..16.02 rows=200 width=60) (actual time=0.025..0.029 rows=13 loops=1)
         Group Key: yearid, schoolid
         Batches: 1  Memory Usage: 40kB
         ->  Seq Scan on inducted_hof_ca_mat  (cost=0.00..12.30 rows=230 width=52) (actual time=0.006..0.008 rows=23 loops=1)
 Planning Time: 0.363 ms
 Execution Time: 0.256 ms
(9 rows)



In [46]:
with_view_cost = 519.45
with_view_timing = 5.882

In [47]:
with_materialized_view_cost = 24.17
with_materialized_view_timing = 0.196

In [48]:
grader.check("q1eiii")

q1eiii results: All test cases passed!

#### Question 1eiv:
Given your findings from inspecting the query plans, as well as your understanding of views and materialized views from lectures, answer the following question. Assign the variable `q1e_part4` to a list of all statements which are true.

1. Consider the following statements:
    1. Views will reduce the execution time and the cost of a query.
    1. Views will reduce the execution time of a query, but not the cost.
    1. Views will reduce the cost of a query, but not the execution time.
    1. Materialized views reduce the execution time and the cost of a query.
    1. Materialized views reduce the execution time, but not cost of a query.
    1. Materialized views reduce the cost of a query, but not the execution time.
    1. Materialized views will result in the same query plan as a query using views.
    1. Materialized views and views take the same time to create.
    1. Materialized views take less time to create than a view.
    1. Materialized views take more time to create than a view.
    
*Note:* Your answer should look like `q1e_part4 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1eiv
points: 2
-->

In [49]:
q1e_part4 = ['D', 'J']  

In [50]:
grader.check("q1eiv")

q1eiv results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 1ev:

Explain your answer to the previous part based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q1ev
manual: true
points: 2
-->

Comparing the query plan from Views and Materialized Views, we see that not only the Cost and Execution Time of Materialized View are significantly lower than Views, but the Planning Time is also lower for Materialized View. This is mainly due to the fact that Views are computed each time that you need to use the view. On the other hand, Materialized Views are actually stored so there may be a higher cost of maintenance when base tables are updated, but then the query executions are faster as you don't need to re-compute materialized views on demand.

<!-- END QUESTION -->



## Question 2: Predicate Pushdown
In this question, we will explore the impact of predicates on a query's execution, particularly inspecting when the optimizer applies predicates.

### Question 2a:
Recall the `inducted_hof_ca` created in `Question 1c`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost.

In [51]:
query_view_str = "SELECT * FROM inducted_hof_ca;"
!psql -h localhost -d baseball -c "explain analyze $query_view_str"

                                                                 QUERY PLAN                                                                  
---------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=386.71..527.38 rows=33 width=33) (actual time=6.079..6.620 rows=23 loops=1)
   Join Filter: ((h.playerid)::text = (p.playerid)::text)
   ->  Hash Join  (cost=386.42..485.79 rows=96 width=30) (actual time=6.026..6.369 rows=23 loops=1)
         Hash Cond: ((h.playerid)::text = (c.playerid)::text)
         ->  Seq Scan on halloffame h  (cost=0.00..96.39 rows=323 width=13) (actual time=0.021..0.666 rows=323 loops=1)
               Filter: ((inducted)::text = 'Y'::text)
               Rows Removed by Filter: 3868
         ->  Hash  (cost=361.99..361.99 rows=1955 width=17) (actual time=5.632..5.635 rows=2948 loops=1)
               Buckets: 4096 (originally 2048)  Batches: 1 (originally 1)  Me

In [52]:
query_view_cost = 527.38
query_view_timing = 5.862

In [53]:
grader.check("q2a")

q2a results: All test cases passed!

### Question 2b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [54]:
%%sql result_2b <<
SELECT * 
FROM inducted_hof_ca
WHERE yearid > 2010;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8 rows affected.
Returning data to local variable result_2b


In [55]:
query_view_with_filter_str = "SELECT * FROM inducted_hof_ca WHERE yearid > 2010;"
!psql -h localhost -d baseball -c "explain analyze $query_view_with_filter_str"

                                                                          QUERY PLAN                                                                           
---------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.85..209.84 rows=2 width=33) (actual time=0.542..0.879 rows=8 loops=1)
   Join Filter: ((h.playerid)::text = (p.playerid)::text)
   ->  Nested Loop  (cost=0.56..207.24 rows=6 width=30) (actual time=0.516..0.800 rows=8 loops=1)
         ->  Nested Loop  (cost=0.29..190.63 rows=55 width=30) (actual time=0.497..0.699 rows=27 loops=1)
               ->  Seq Scan on halloffame h  (cost=0.00..106.86 rows=21 width=13) (actual time=0.444..0.489 rows=31 loops=1)
                     Filter: ((yearid > 2010) AND ((inducted)::text = 'Y'::text))
                     Rows Removed by Filter: 4160
               ->  Index Only Scan using collegeplaying_pkey on colle

In [56]:
query_view_with_filter_cost = 209.84
query_view_with_filter_timing = 0.974

In [57]:
grader.check("q2b")

q2b results: All test cases passed!

### Question 2c:
Given your findings from inspecting the query plans of queries from `Question 2a-b`, answer the following question. Assign the variable `q2c` to a list of all statements which are true.

1. Consider the following statements:
    1. Adding a filter lowered the cost.
    1. Adding a filter increased the cost.
    1. Adding a filter did not change the cost.
    1. Adding a filter increased the execution time.
    1. Adding a filter decreased the execution time.
    1. Adding a filter did not change the execution time.
    1. No statement is true.
    
    
**Note:** Your answer should look like `q2c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [58]:
q2c = ['A','E']

In [59]:
grader.check("q2c")

q2c results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 2d:

Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 2
-->

Looking at the query plan result, we see that adding a filter lowered both the cost and time. This is because with the added filtering clause, the query optimizer execute the query using predicate pushdown, adding it to the query in inducted_hof_ca (Question 1c). So it filters yearid prior to joining, significantly reducing the amount of rows we need to deal with.

<!-- END QUESTION -->



## Question 3: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

### Question 3a:
Perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In [60]:
%%sql q3a_join <<
SELECT *
FROM people
INNER JOIN collegeplaying ON people.playerid = collegeplaying.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3a_join


Inspect the query plan for the query you wrote above.

In [61]:
q3a_query_str = "SELECT * FROM people INNER JOIN collegeplaying ON people.playerid = collegeplaying.playerid;"
!psql -h localhost -d baseball -c "explain analyze $q3a_query_str"

                                                       QUERY PLAN                                                        
-------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..1193.88 rows=17350 width=167) (actual time=8.831..18.973 rows=17350 loops=1)
   Hash Cond: ((collegeplaying.playerid)::text = (people.playerid)::text)
   ->  Seq Scan on collegeplaying  (cost=0.00..286.50 rows=17350 width=21) (actual time=0.018..1.263 rows=17350 loops=1)
   ->  Hash  (cost=619.70..619.70 rows=19370 width=146) (actual time=8.574..8.577 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 3633kB
         ->  Seq Scan on people  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.016..1.714 rows=19370 loops=1)
 Planning Time: 1.269 ms
 Execution Time: 19.726 ms
(8 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3a` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3a = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [62]:
q3a = ['C']

In [63]:
grader.check("q3a")

q3a results: All test cases passed!

### Question 3b

Similar to Question 3a, perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In addition, **sort your output by `playerid`.**

In [64]:
%%sql q3b_join <<
SELECT *
FROM people
INNER JOIN collegeplaying ON people.playerid = collegeplaying.playerid
ORDER BY people.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3b_join


Inspect the query plan for the query you wrote above.

In [65]:
q3b_query_str = "SELECT * FROM people INNER JOIN collegeplaying ON people.playerid = collegeplaying.playerid ORDER BY people.playerid;"
!psql -h localhost -d baseball -c "explain analyze $q3b_query_str"

                                                                        QUERY PLAN                                                                        
----------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..1910.20 rows=17350 width=167) (actual time=0.041..18.352 rows=17350 loops=1)
   Merge Cond: ((people.playerid)::text = (collegeplaying.playerid)::text)
   ->  Index Scan using master_pkey on people  (cost=0.29..1024.36 rows=19370 width=146) (actual time=0.017..4.207 rows=19368 loops=1)
   ->  Index Only Scan using collegeplaying_pkey on collegeplaying  (cost=0.29..620.54 rows=17350 width=21) (actual time=0.019..2.360 rows=17350 loops=1)
         Heap Fetches: 0
 Planning Time: 1.103 ms
 Execution Time: 19.083 ms
(7 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3b` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3b = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [66]:
q3b = ['B']

In [67]:
grader.check("q3b")

q3b results: All test cases passed!

### Question 3c
Write a query to retrieve all possible player pair combinations. Project all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [68]:
%%sql q3c_join <<
SELECT *
FROM people p1
INNER JOIN people p2 ON p2.playerid != p1.playerid
LIMIT 1000;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1000 rows affected.
Returning data to local variable q3c_join


In [69]:
q3c_join

playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,deathcountry,deathstate,deathcity,namefirst,namelast,namegiven,weight,height,bats,throws,debut,finalgame,retroid,bbrefid,playerid_1,birthyear_1,birthmonth_1,birthday_1,birthcountry_1,birthstate_1,birthcity_1,deathyear_1,deathmonth_1,deathday_1,deathcountry_1,deathstate_1,deathcity_1,namefirst_1,namelast_1,namegiven_1,weight_1,height_1,bats_1,throws_1,debut_1,finalgame_1,retroid_1,bbrefid_1
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aaronha01,1934,2,5,USA,AL,Mobile,None,None,None,None,None,None,Hank,Aaron,Henry Louis,180,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aaronto01,1939,8,5,USA,AL,Mobile,1984,8,16,USA,GA,Atlanta,Tommie,Aaron,Tommie Lee,190,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aasedo01,1954,9,8,USA,CA,Orange,None,None,None,None,None,None,Don,Aase,Donald William,190,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,abadan01,1972,8,25,USA,FL,Palm Beach,None,None,None,None,None,None,Andy,Abad,Fausto Andres,184,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,abadfe01,1985,12,17,D.R.,La Romana,La Romana,None,None,None,None,None,None,Fernando,Abad,Fernando Antonio,220,73.0,L,L,2010-07-28,2017-10-01,abadf001,abadfe01
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,abadijo01,1850,11,4,USA,PA,Philadelphia,1905,5,17,USA,NJ,Pemberton,John,Abadie,John W.,192,72.0,R,R,1875-04-26,1875-06-10,abadj101,abadijo01
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,abbated01,1877,4,15,USA,PA,Latrobe,1957,1,6,USA,FL,Fort Lauderdale,Ed,Abbaticchio,Edward James,170,71.0,R,R,1897-09-04,1910-09-15,abbae101,abbated01
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,abbeybe01,1869,11,11,USA,VT,Essex,1962,6,11,USA,VT,Colchester,Bert,Abbey,Bert Wood,175,71.0,R,R,1892-06-14,1896-09-23,abbeb101,abbeybe01
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,abbeych01,1866,10,14,USA,NE,Falls City,1926,4,27,USA,CA,San Francisco,Charlie,Abbey,Charles S.,169,68.0,L,L,1893-08-16,1897-08-19,abbec101,abbeych01
aardsda01,1981,12,27,USA,CO,Denver,None,None,None,None,None,None,David,Aardsma,David Allan,215,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,abbotda01,1862,3,16,USA,OH,Portage,1930,2,13,USA,MI,Ottawa Lake,Dan,Abbott,Leander Franklin,190,71.0,R,R,1890-04-19,1890-05-23,abbod101,abbotda01


Inspect the query plan for the query you wrote above.

In [70]:
q3c_query_str = "SELECT * FROM people p1 INNER JOIN people p2 ON p2.playerid != p1.playerid LIMIT 1000;"
!psql -h localhost -d baseball -c "explain analyze $q3c_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Limit  (cost=0.00..15.00 rows=1000 width=292) (actual time=0.034..1.004 rows=1000 loops=1)
   ->  Nested Loop  (cost=0.00..5629241.33 rows=375177530 width=292) (actual time=0.032..0.933 rows=1000 loops=1)
         Join Filter: ((p2.playerid)::text <> (p1.playerid)::text)
         Rows Removed by Join Filter: 1
         ->  Seq Scan on people p1  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.017..0.017 rows=1 loops=1)
         ->  Materialize  (cost=0.00..716.55 rows=19370 width=146) (actual time=0.008..0.501 rows=1001 loops=1)
               ->  Seq Scan on people p2  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.001..0.093 rows=1001 loops=1)
 Planning Time: 0.866 ms
 Execution Time: 1.186 ms
(9 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3c` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [71]:
q3c = ['A']

In [72]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3d
Please discuss your findings in the previous parts. In particular, we are interested in hearing why you think the query optimizer chooses the ultimate join approach in each of the above three scenarios. Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 3
-->

The query in 3a utilized hash join could be because it doesn't require any sorting and results in a relatively linear runtime, so we can just assign each tuples into a bucket and join them. 

3b utilized merge join because our output has to be sorted, and a sort-merge join can effectively sort our query piror to joining.

3c utilized nest loop join because we have to compare each player with another player in the query to create a pair combination and there are no other ways to do this directly. Also since we're limiting the ouput to 1000, it's reasonbly sized to perform nest loop join.

<!-- END QUESTION -->



## Question 4: Indexes

### Question 4a:
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [73]:
%%sql
SELECT *
FROM salaries
LIMIT 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


yearid,teamid,lgid,playerid,salary
1985,ATL,NL,barkele01,870000.0
1985,ATL,NL,bedrost01,550000.0
1985,ATL,NL,benedbr01,545000.0
1985,ATL,NL,campri01,633333.0
1985,ATL,NL,ceronri01,625000.0
1985,ATL,NL,chambch01,800000.0
1985,ATL,NL,dedmoje01,150000.0
1985,ATL,NL,forstte01,483333.0
1985,ATL,NL,garbege01,772000.0
1985,ATL,NL,harpete01,250000.0


In [74]:
%%sql
SELECT *
FROM appearances
LIMIT 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


yearid,teamid,lgid,playerid,g_all,gs,g_batting,g_defense,g_p,g_c,g_1b,g_2b,g_3b,g_ss,g_lf,g_cf,g_rf,g_of,g_dh,g_ph,g_pr
1871,TRO,NA,abercda01,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0
1871,RC1,NA,addybo01,25,25,25,25,0,0,0,22,0,3,0,0,0,0,0,0,0
1871,CL1,NA,allisar01,29,29,29,29,0,0,0,2,0,0,0,29,0,29,0,0,0
1871,WS3,NA,allisdo01,27,27,27,27,0,27,0,0,0,0,0,0,0,0,0,0,0
1871,RC1,NA,ansonca01,25,25,25,25,0,5,1,2,20,0,1,0,0,1,0,0,0
1871,FW1,NA,armstbo01,12,12,12,12,0,0,0,0,0,0,0,11,1,12,0,0,0
1871,RC1,NA,barkeal01,1,1,1,1,0,0,0,0,0,0,1,0,0,1,0,0,0
1871,BS1,NA,barnero01,31,31,31,31,0,0,0,16,0,15,0,0,0,0,0,0,0
1871,FW1,NA,barrebi01,1,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0
1871,BS1,NA,barrofr01,18,17,18,18,0,0,0,1,0,0,13,0,4,17,0,0,0


In [75]:
%%sql result_4a <<
SELECT playerid, AVG(salary) as avg_salary
FROM salaries
NATURAL JOIN appearances
WHERE g_batting = 10
GROUP BY playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
134 rows affected.
Returning data to local variable result_4a


In [76]:
result_4a

playerid,avg_salary
anderla02,240000.0
ashbyan01,109000.0
ayraubo01,125000.0
bankswi01,240000.0
batchri01,157500.0
batiski01,100000.0
batismi01,153000.0
bautijo01,150000.0
beller01,145000.0
benesan01,68000.0


In [77]:
# Do not delete/edit this cell
result_4a.DataFrame().sort_values('playerid').to_csv('results/result_4a.csv', index=False)

In [78]:
grader.check("q4ai")

q4ai results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [79]:
result_4a_query_str = "SELECT playerid, AVG(salary) as avg_salary FROM salaries NATURAL JOIN appearances WHERE g_batting = 10 GROUP BY playerid;"
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3636.26..3636.29 rows=1 width=17) (actual time=21.889..21.958 rows=134 loops=1)
   Group Key: salaries.playerid
   ->  Sort  (cost=3636.26..3636.27 rows=1 width=17) (actual time=21.877..21.888 rows=138 loops=1)
         Sort Key: salaries.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2899.48..3636.25 rows=1 width=17) (actual time=13.280..21.692 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.te

In [80]:
result_4a_cost = 3637.39
result_4a_timing = 22.572

In [81]:
grader.check("q4aii")

q4aii results: All test cases passed!

### Question 4b:
Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [82]:
%%sql result_4b <<
CREATE INDEX g_batting_idx
ON appearances(g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4b


Now, re-inspect the query plan of the query from `Question 4a` and record its execution time and cost.

In [83]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=2369.03..2369.05 rows=1 width=17) (actual time=11.294..11.367 rows=134 loops=1)
   Group Key: salaries.playerid
   ->  Sort  (cost=2369.03..2369.04 rows=1 width=17) (actual time=11.281..11.293 rows=138 loops=1)
         Sort Key: salaries.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=1632.25..2369.02 rows=1 width=17) (actual time=3.183..11.106 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.tea

In [84]:
result_4b_cost = 2385.23
result_4b_timing = 11.670

In [85]:
grader.check("q4b")

q4b results: All test cases passed!

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, **drop the index before moving onto the next question.**

In [86]:
%%sql 
DROP INDEX g_batting_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4c:
Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table.

In [87]:
%%sql result_4c <<
CREATE INDEX salary_idx
ON salaries(salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4c


Re-inspect the query plan of the query from the `Question 4a` and record its execution time and cost.

In [88]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                     
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3636.26..3636.29 rows=1 width=17) (actual time=21.954..22.023 rows=134 loops=1)
   Group Key: salaries.playerid
   ->  Sort  (cost=3636.26..3636.27 rows=1 width=17) (actual time=21.942..21.952 rows=138 loops=1)
         Sort Key: salaries.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2899.48..3636.25 rows=1 width=17) (actual time=13.518..21.746 rows=138 loops=1)
               Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.te

In [89]:
result_4a_cost = 3637.39    #no index of batting
result_4a_timing = 22.572

In [90]:
result_4b_cost = 2385.23   #index of batting
result_4b_timing = 11.670

In [91]:
result_4c_cost = 3637.39     #index of salary
result_4c_timing = 23.465

In [92]:
grader.check("q4c")

q4c results: All test cases passed!

### Question 4d:
#### Question 4di:
Given your findings from inspecting the query plans with no indexes, an index on `g_batting`, and an index on `salary`, answer the following question. Assign the variable `q4d_part1` to a list of all options which are true for the proposed statement.

1. Consider the following statements:
    1. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.
    1. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `g_batting` index did have a significant impact on the query cost and execution time.
    1. Adding the `salary` index did not have a significant impact on the query execution time and cost.
    1. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as follows: `q4d_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4di
points: 2
-->

In [93]:
q4d_part1 = ['D', 'E']

In [94]:
grader.check("q4di")

q4di results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4dii:

Explain your answer based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4dii
manual: true
points: 2
-->

Adding the g_batting index did have a significant improved the query cost and execution time because by indexing on batting, we are able to more quickly filter out players that batted more or less than 10 games.

Adding the salary index did not have a significant impact on the query execution time and cost because we're just taking the average values of salary from each player, adding index to the salaries would not make much difference on the computation.

<!-- END QUESTION -->



In the following question, we will further explore the impact of indexes on query performance. To avoid any interference by the `salary_idx` index, please drop the index before moving onto the next question.

In [95]:
%%sql 
DROP INDEX salary_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4e:
#### Question 4ei:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games and batted in 10 games (the number of games in which a player played can be found in the `g_all` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [96]:
%%sql result_4ei <<
SELECT playerid, yearid, salary
FROM salaries
NATURAL JOIN appearances
WHERE g_batting = 10 AND g_all = 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
120 rows affected.
Returning data to local variable result_4ei


In [97]:
# Do not delete/edit this cell
result_4ei.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4ei.csv', index=False)

In [98]:
grader.check("q4ei_part1")

q4ei_part1 results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [99]:
result_4ei_query_str = "SELECT playerid, yearid, salary FROM salaries NATURAL JOIN appearances WHERE g_batting = 10 AND g_all = 10;"
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                QUERY PLAN                                                                                                 
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..3296.09 rows=1 width=21) (actual time=9.251..16.532 rows=120 loops=1)
   ->  Seq Scan on appearances  (cost=0.00..3133.84 rows=20 width=20) (actual time=0.023..13.119 rows=1289 loops=1)
         Filter: ((g_batting = 10) AND (g_all = 10))
         Rows Removed by Filter: 102967
   ->  Index Scan using salaries_pkey on salaries  (cost=0.29..8.11 rows=1 width=28) (actual time=0.002..0.002 rows=0 loops=1289)
         Index Cond: ((yearid = appearances.yearid) AND ((teamid)::text = (appearances.teamid)::text) AND ((lgid)::text = (appearances.lgid)::text) AND ((pla

In [100]:
result_4ei_cost = 3300.40
result_4ei_timing = 17.227

In [101]:
grader.check("q4ei_part2")

q4ei_part2 results: All test cases passed!

#### Question 4eii:
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the query from `Question 4ei` and record the execution time and cost.

In [102]:
%%sql result_4eii << 
CREATE INDEX g_batting_idx_4eii
ON appearances(g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eii


In [103]:
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                QUERY PLAN                                                                                                 
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=18.44..1771.18 rows=1 width=21) (actual time=2.926..5.817 rows=120 loops=1)
   ->  Bitmap Heap Scan on appearances  (cost=18.15..1608.93 rows=20 width=20) (actual time=0.364..2.573 rows=1289 loops=1)
         Recheck Cond: (g_batting = 10)
         Filter: (g_all = 10)
         Rows Removed by Filter: 58
         Heap Blocks: exact=899
         ->  Bitmap Index Scan on g_batting_idx_4eii  (cost=0.00..18.15 rows=1314 width=0) (actual time=0.235..0.236 rows=1347 loops=1)
               Index Cond: (g_batting = 10)
   ->  Index Scan using salaries_pkey on salaries  (co

In [104]:
result_4eii_with_index_cost = 1790.69
result_4eii_with_index_timing = 9.003

In [105]:
grader.check("q4eii")

q4eii results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [106]:
%%sql 
DROP INDEX g_batting_idx_4eii;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4eiii:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [107]:
%%sql result_4eiii <<
SELECT playerid, yearid, salary
FROM salaries
NATURAL JOIN appearances
WHERE g_batting = 10 OR g_all = 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
216 rows affected.
Returning data to local variable result_4eiii


In [108]:
# Do not delete/edit this cell
result_4eiii.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4eiii.csv', index=False)

In [109]:
grader.check("q4eiii_part1")

q4eiii_part1 results: All test cases passed!

In [110]:
result_4eiii_query_str = "SELECT playerid, yearid, salary FROM salaries NATURAL JOIN appearances WHERE g_batting = 10 OR g_all = 10;"
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                               
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3190.72..3927.49 rows=1 width=21) (actual time=14.346..22.125 rows=216 loops=1)
   Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.014..1.866 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2844 width=20) (actual time=14.106..14.108 rows=

In [111]:
result_4eiii_cost = 3930.71
result_4eiii_timing = 21.018 

In [112]:
grader.check("q4eiii_part2")

q4eiii_part2 results: All test cases passed!

#### Question 4eiv
Now, let's see the impact of adding an index on `g_batting` column will have on the query. Re-create the index and re-inspect the query from `Question 4eiii` and record the execution time and cost.

In [113]:
%%sql result_4eiv << 
CREATE INDEX g_batting_idx_4eiv
ON appearances(g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eiv


In [114]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                               
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3190.72..3927.49 rows=1 width=21) (actual time=14.795..23.404 rows=216 loops=1)
   Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.016..2.025 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2844 width=20) (actual time=14.482..14.485 rows=

In [115]:
result_4eiv_with_index_cost = 3930.71
result_4eiv_with_index_timing = 21.253 

In [116]:
grader.check("q4eiv")

q4eiv results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [117]:
%%sql 
DROP INDEX g_batting_idx_4eiv

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4ev:
Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost.

In [118]:
%%sql result_4v <<
CREATE INDEX g_batting_g_all_idx
ON appearances(g_batting, g_all);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4v


In [119]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                               
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=2883.88..3620.65 rows=1 width=21) (actual time=4.824..13.058 rows=216 loops=1)
   Hash Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=28) (actual time=0.015..2.011 rows=26428 loops=1)
   ->  Hash  (cost=2827.00..2827.00 rows=2844 width=20) (actual time=4.607..4.612 rows=165

In [120]:
result_4ev_multiple_col_index_cost = 3608.10
result_4ev_multiple_col_index_timing = 11.875

In [121]:
grader.check("q4ev")

q4ev results: All test cases passed!

#### Question 4evi:
Given your findings from inspecting the query plans from `Question 4e`, answer the following question. Assign the variable `q4e_part6` to a list of all statements that are true.

1. Consider the following statements:
    1. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as follows: `q4e_part6 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4evi
points: 2
-->

In [122]:
result_4ei_cost = 3300.40               #AND no index
result_4ei_timing = 17.227

result_4eii_with_index_cost = 1790.69     #AND w/ index on g_batting
result_4eii_with_index_timing = 9.003

result_4eiii_cost = 3930.71         # OR no index
result_4eiii_timing = 21.018

result_4eiv_with_index_cost = 3930.71     # OR w/ index on g_batting
result_4eiv_with_index_timing = 21.253 

result_4ev_multiple_col_index_cost = 3608.10      # OR w/ g_batting & g_all
result_4ev_multiple_col_index_timing = 11.875

In [123]:
q4e_part6 = ['C', 'I']

In [175]:
grader.check("q4evi")

q4evi results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4evii

Explain your answer to the previous part based on your knowledge from lectures, and details of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4evii
manual: true
points: 2
-->

Looking at the query plan results, we see that adding an index on a column used in an AND predicate reduced the query cost and the execution time, because with g_batting indexed, SQL only need to look at players that batted exactly 10 games, since most of them evaluated to `False` in our dataset, SQL doesn't have to check g_all. We also see that adding an index on a column used in an OR predicate results in essentially the same cost and time. 
This could be because even with g_batting indexed, SQL would still have to check the other pages associated to g_all. While with an AND predicate, SQL only need to check for pages of g_batting that are `False`, which makes things faster. Therefore, we see that multicolumn index on g_batting and g_all lowered the cost and time significantly by avoid looking through all the rows of both columns. Thus option I is chosen in addition to option C.

<!-- END QUESTION -->



## Question 5: More Indexes
### Question 5a:
Write 2 queries: one that finds the minimum salary from the salary table and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

In [125]:
%%sql
SELECT *
FROM salaries
LIMIT 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


yearid,teamid,lgid,playerid,salary
1985,ATL,NL,barkele01,870000.0
1985,ATL,NL,bedrost01,550000.0
1985,ATL,NL,benedbr01,545000.0
1985,ATL,NL,campri01,633333.0
1985,ATL,NL,ceronri01,625000.0
1985,ATL,NL,chambch01,800000.0
1985,ATL,NL,dedmoje01,150000.0
1985,ATL,NL,forstte01,483333.0
1985,ATL,NL,garbege01,772000.0
1985,ATL,NL,harpete01,250000.0


In [126]:
%%sql result_5a_min << 
SELECT MIN(salary)
FROM salaries;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_min


In [127]:
result_5a_min_query_str = 'SELECT MIN(salary) FROM salaries;'
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=7.717..7.719 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.022..3.251 rows=26428 loops=1)
 Planning Time: 0.722 ms
 Execution Time: 7.848 ms
(4 rows)



In [128]:
result_5a_min_query_cost = 525.36
result_5a_min_query_timing = 5.573

In [129]:
grader.check("q5ai")

q5ai results: All test cases passed!

In [130]:
%%sql result_5a_avg <<
SELECT AVG(salary)
FROM salaries;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_avg


In [131]:
result_5a_avg_query_str = 'SELECT AVG(salary) FROM salaries;'
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=6.561..6.563 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.011..2.481 rows=26428 loops=1)
 Planning Time: 0.326 ms
 Execution Time: 6.630 ms
(4 rows)



In [132]:
result_5a_avg_query_cost = 525.36 
result_5a_avg_query_timing = 5.144 

In [133]:
grader.check("q5aii")

q5aii results: All test cases passed!

### Question 5b:
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [134]:
%%sql 
CREATE INDEX salary_idx_5b
ON salaries(salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [135]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                                   QUERY PLAN                                                                    
-------------------------------------------------------------------------------------------------------------------------------------------------
 Result  (cost=0.32..0.33 rows=1 width=8) (actual time=0.069..0.070 rows=1 loops=1)
   InitPlan 1 (returns $0)
     ->  Limit  (cost=0.29..0.32 rows=1 width=8) (actual time=0.066..0.066 rows=1 loops=1)
           ->  Index Only Scan using salary_idx_5b on salaries  (cost=0.29..762.78 rows=26428 width=8) (actual time=0.064..0.065 rows=1 loops=1)
                 Index Cond: (salary IS NOT NULL)
                 Heap Fetches: 0
 Planning Time: 0.654 ms
 Execution Time: 0.120 ms
(8 rows)



In [136]:
result_5b_min_query_cost = 0.33 
result_5b_min_query_timing = 0.123 

In [137]:
grader.check("q5bi")

q5bi results: All test cases passed!

In [138]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=5.419..5.422 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.012..2.075 rows=26428 loops=1)
 Planning Time: 0.471 ms
 Execution Time: 5.519 ms
(4 rows)



In [139]:
result_5b_avg_query_cost = 525.36 
result_5b_avg_query_timing = 5.548 

In [140]:
grader.check("q5bii")

q5bii results: All test cases passed!

In the following questions, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `salary` column, drop the index before moving onto the next question.

In [141]:
%%sql 
DROP INDEX salary_idx_5b;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 5c:
Given your findings from `Question 5a-b`, answer the following question. Assign the variable `q5c` to the correct answer choice.

1. Which of the following statements is true?
    1. An index on the column being aggregated in a query will always provide a performance enhancement.
    1. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
    1. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
    1. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

*Note:* Your answer should be formatted as follows: `q5c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q5c
points: 1
-->

In [142]:
result_5a_min_query_cost = 525.36
result_5a_min_query_timing = 5.573                # min no index

result_5a_avg_query_cost = 525.36 
result_5a_avg_query_timing = 5.144                # avg no index

result_5b_min_query_cost = 0.33 
result_5b_min_query_timing = 0.123                # min index

result_5b_avg_query_cost = 525.36 
result_5b_avg_query_timing = 5.548                # avg index

In [143]:
q5c = ['D']

In [144]:
grader.check("q5c")

q5c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 5d:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 2
-->

Looking at the query plan results, we see that finding the MIN benefited from indexing, while finding the AVG did not. We know that finding the MAX will also benefit because once we indexed salaries, SQL can quickly grab the highest/lowest value based on the index, like the left most and right most value in a tree, without having to go through all of the values. Finding the AVG and COUNT on the other hand, will always require us to compute the same amount to find the average number and also count through each tuple to get the count.

<!-- END QUESTION -->



## Question 6: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.

### Question 6a
Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [145]:
%%sql
SELECT *
FROM batting
LIMIT 10;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10 rows affected.


playerid,yearid,stint,teamid,lgid,g,ab,r,h,h2b,h3b,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
abercda01,1871,1,TRO,NA,1,4,0,0,0,0,0,0,0,0,0,0,None,None,None,None,0
addybo01,1871,1,RC1,NA,25,118,30,32,6,0,0,13,8,1,4,0,None,None,None,None,0
allisar01,1871,1,CL1,NA,29,137,28,40,4,5,0,19,3,1,2,5,None,None,None,None,1
allisdo01,1871,1,WS3,NA,27,133,28,44,10,2,2,27,1,1,0,2,None,None,None,None,0
ansonca01,1871,1,RC1,NA,25,120,29,39,11,3,0,16,6,2,2,1,None,None,None,None,0
armstbo01,1871,1,FW1,NA,12,49,9,11,2,1,0,5,0,1,0,1,None,None,None,None,0
barkeal01,1871,1,RC1,NA,1,4,0,1,0,0,0,2,0,0,1,0,None,None,None,None,0
barnero01,1871,1,BS1,NA,31,157,66,63,10,9,0,34,11,6,13,1,None,None,None,None,1
barrebi01,1871,1,FW1,NA,1,5,1,1,1,0,0,1,0,0,0,0,None,None,None,None,0
barrofr01,1871,1,BS1,NA,18,86,13,13,2,1,0,11,1,0,0,0,None,None,None,None,0


In [146]:
%%sql result_6a <<
SELECT playerid, yearid, teamid, ab
FROM batting
WHERE ab > 500;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8839 rows affected.
Returning data to local variable result_6a


In [147]:
# Do not delete/edit this cell
result_6a.DataFrame().sort_values(['playerid', 'yearid', 'teamid']).iloc[:1000].to_csv('results/result_6a.csv', index=False)

In [148]:
grader.check("q6ai")

q6ai results: All test cases passed!

In [149]:
result_6a_query_str = 'SELECT playerid, yearid, teamid, ab FROM batting WHERE ab > 500;'
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2884.05 rows=9027 width=21) (actual time=0.302..14.343 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.365 ms
 Execution Time: 14.719 ms
(5 rows)



In [150]:
result_6a_cost = 2884.05 
result_6a_timing = 15.561 

In [151]:
grader.check("q6aii")

q6aii results: All test cases passed!

### Question 6b:
Cluster the `batting` table on its primary key (hint: use `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it. Re-inspect the query plan for the query from `Question 6a` and record the execution time and cost.

In [152]:
%sql \di

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
27 rows affected.


Schema,Name,Type,Owner
public,allstarfull_pkey,index,jovyan
public,appearances_pkey,index,jovyan
public,awardsmanagers_pkey,index,jovyan
public,awardsplayers_pkey,index,jovyan
public,awardssharemanagers_pkey,index,jovyan
public,awardsshareplayers_pkey,index,jovyan
public,batting_pkey,index,jovyan
public,battingpost_pkey,index,jovyan
public,collegeplaying_pkey,index,jovyan
public,fielding_pkey,index,jovyan


In [153]:
%%sql 
CLUSTER batting USING batting_pkey;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [154]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2878.05 rows=9027 width=21) (actual time=0.015..14.891 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.461 ms
 Execution Time: 15.260 ms
(5 rows)



In [155]:
result_6b_cost = 2878.05 
result_6b_timing = 17.245 

In [156]:
grader.check("q6b")

q6b results: All test cases passed!

### Question 6c
Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [157]:
%%sql 
CREATE INDEX ab_idx
ON batting(ab);

CLUSTER batting USING ab_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.


[]

In [158]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                      QUERY PLAN                                                       
-----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on batting  (cost=106.25..1794.09 rows=9027 width=21) (actual time=0.312..2.042 rows=8839 loops=1)
   Recheck Cond: (ab > 500)
   Heap Blocks: exact=135
   ->  Bitmap Index Scan on ab_idx  (cost=0.00..104.00 rows=9027 width=0) (actual time=0.281..0.281 rows=8839 loops=1)
         Index Cond: (ab > 500)
 Planning Time: 0.511 ms
 Execution Time: 2.417 ms
(7 rows)



In [159]:
result_6c_cost = 1787.36 
result_6c_timing = 2.457 

In [160]:
grader.check("q6c")

q6c results: All test cases passed!

### Question 6d
Given your findings from inspecting the query plans from `Question 6a-c`, answer the following question. Assign the variable `q6d` to a list of all statements that are true.

1. Consider the following statements:
    1. Clustering based on the `ab_idx` decreased the cost of the query.
    1. Clustering based on the `ab_idx` increased the cost of the query.
    1. Clustering based on the `ab_idx` increased the execution time of the query.
    1. Clustering based on the `ab_idx` decreased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the execution time of the query.
    1. None of the above
    
**Note:** Your answer should be formatted as follows: `q6d = ['A', 'B']`

<!--
BEGIN QUESTION
name: q6d
points: 2
-->

In [161]:
result_6a_cost = 2884.05 
result_6a_timing = 15.561   #no clustering

result_6b_cost = 2878.05 
result_6b_timing = 17.245    #clustering on batting_pkey

result_6c_cost = 1787.36 
result_6c_timing = 2.457    #clustering on ab_idx

In [162]:
q6d = ['A', 'D']  

In [163]:
grader.check("q6d")

q6d results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 6e:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q6e
manual: true
points: 2
-->

Looking at the query plan results, we see that only clustering based on ab_idx decreased the cost and time compared to no clustering, while clustering based on batting_pkey actually didn't make any significant difference. This could be due to the fact that we're filtering on the ab values, so clustering based on batting_pkey would not help with this WHERE clause as it doesn't organize the ab values. In fact, the overhead of traversing the unnecessary index might even increase the time. On the other hand, clustering based on ab_idx helps because what we're filtering on are implemented with B+ trees, at the leaf level the pointers point to blocks in which tuples are clustered by ab values, making the operation much faster to filter.

<!-- END QUESTION -->



## Question 7: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

### Question 7a:
Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [164]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

Next, run the provided update script and record the **wall time**.

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [165]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 5.74 ms, sys: 754 µs, total: 6.49 ms
Wall time: 3.47 s


[]

In [166]:
result_7a_timing = 3.8

In [167]:
grader.check("q7a")

q7a results: All test cases passed!

Before adding an index to the salaries table and re-timing the updates, delete all the rows that were added to the table from the update script.

In [168]:
%%sql
DELETE FROM salaries
WHERE added = 'true';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
300000 rows affected.


[]

### Question 7b:
Now, create an index on the `salary` column and record the **wall time** after executing the update script.

In [169]:
%%sql 
CREATE INDEX salary_idx_7b
ON salaries(salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of last subpart.

In [170]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 6.37 ms, sys: 1.55 ms, total: 7.92 ms
Wall time: 6.86 s


[]

In [171]:
result_7b_timing = 5.31 

In [172]:
grader.check("q7b")

q7b results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 7c:
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened?

<!--
BEGIN QUESTION
name: q7c
manual: true
points: 2
-->

I noticed that when added an index into the salaries table, the wall time increased by about 1.5 seconds. This may be because we are inserting 300,000 rows, a pretty large amount into the salaries table. This means that we are constantly triggering regular updates to our tables as it's managing and updating the indexes due to the newly inserted rows. This could take up a lot of resource and time given such a large amount of rows of data introduced. 

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 8: Takeaway

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 methods you learned in this project. The method can be either the optimization done by the database system, or the fine tuning done by the user.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

<!--
BEGIN QUESTION
name: q8
manual: true
points: 6
-->

. One method that I learned is to index and cluster only on the column that you plan on ordering/filtering, but not on other columns. As wee have seen in previous questions, indexing and clustering on the right columns can significantly reduce the cost and time in query plans as SQL can lookup values in an organized fashion (B+ Trees etc.). However, if we do this on unnecessary columns, it might introduce overhead. In Q7 for instance, creating index on columns when you are inserting a large amount of new tuples into the relation slowed down the wall time significantly. But when we apply multicolumn index on the columns that we're predicating on in an OR predicate, it improved the optimization significantly. 

. Another method that I learned is the different join approachs that's done by the query optimizer. When it doesn't require any sorting, the optimizer utilizes hash join to assign each tuples into a bucket and just join them as it results in a relatively linearly runtime. When the output need to be sorted however, the optimizer utilizes merge join because a sort-merge join can effectively sort our query piror to joining, cutting down the cost and time significantly. Then a nest loop join is utilized when we have to compare each tuple with another tuple to create a pair combination. There are often no other ways to do this directly and if our dataset is not too large, it's good to perform nest loop join. 

. The third method that I learned is to use Materialized View when we need repeatly issue queries that are variant of one. When comparing the query plan from Views and Materialized Views, we see that not only the Cost and Execution Time of Materialized View are significantly lower than Views, but the Planning Time is also lower for Materialized View. This is mainly due to the fact that Views are computed each time that you need to use the view. On the other hand, Materialized Views are actually stored so there may be a higher cost of maintenance when base tables are updated, but then the query executions are faster as you don't need to re-compute materialized views on demand.

<!-- END QUESTION -->



## Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For submission on Gradescope, you will need to submit BOTH the proj2.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [176]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

updating: results/ (stored 0%)
updating: results/result_1a.csv (deflated 53%)
updating: results/result_1c.csv (deflated 64%)
updating: results/result_1di_view.csv (deflated 37%)
updating: results/result_1di_no_view.csv (deflated 37%)
updating: results/result_1ei.csv (deflated 64%)
updating: results/result_1eii.csv (deflated 37%)
updating: results/result_4a.csv (deflated 59%)
updating: results/result_4ei.csv (deflated 62%)
updating: results/result_4eiii.csv (deflated 64%)
updating: results/result_6a.csv (deflated 72%)


/home/jovyan/fa22/proj/proj2/results.zip

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [177]:
grader.check_all()

q0 results: All test cases passed!

q1a results: All test cases passed!

q1bi results: All test cases passed!

q1bii results: All test cases passed!

q1c results: All test cases passed!

q1di results: All test cases passed!

q1dii results: All test cases passed!

q1ei results: All test cases passed!

q1eii results: All test cases passed!

q1eiii results: All test cases passed!

q1eiv results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q2c results: All test cases passed!

q3a results: All test cases passed!

q3b results: All test cases passed!

q3c results: All test cases passed!

q4ai results: All test cases passed!

q4aii results: All test cases passed!

q4b results: All test cases passed!

q4c results: All test cases passed!

q4di results: All test cases passed!

q4ei_part1 results: All test cases passed!

q4ei_part2 results: All test cases passed!

q4eii results: All test cases passed!

q4eiii_part1 results: All test cases passe

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [178]:
# Save your notebook first, then run this cell to export your submission.
grader.export()